# Navigating the pipeline

Here we get to explore the IBL pipeline and start performing some analysis on it.

First thing first, let's **import the IBL pipeline package**.

In [ ]:
from ibl_pipeline import subject, acquisition, action, behavior, reference
from ibl_pipeline.analyses.behavior import PsychResults
import datajoint as dj
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Each of these modules correspond to a DataJoint **schema** -- or a collection of related tables.

In [ ]:
subject.Subject()

DataJoint Graph offers a nice way to visualize the pipeline. 

**NOTE: this may not work if you don't have Graphviz installed on your system.**

In [ ]:
dj.Diagram(subject)

## Analyzing existing data

Let's take a look at the trial set with some minimum number of trials

In [ ]:
trial_set_keys = (behavior.TrialSet() & 'n_trials > 600').fetch('KEY')

In [ ]:
len(trial_set_keys)

And not focus on one such trial set

In [ ]:
key = trial_set_keys[0]
key

In [ ]:
behavior.TrialSet & key

and all trials falling under that particular trial set

In [ ]:
behavior.TrialSet.Trial()

### Computing some statistics

Here let's compute some statistics on the trials

### Turning it into a table

To define a new table for your own, start by creating a schema in which you'll place your new table. Most users are given privileges to create any schema starting with their username followed by `_`, as in:

In [ ]:
schema = dj.schema('user_eywalker_tutorial')

Let's define a new computed table that would store the computed statistics.

## Analyzing psychometric curves

We now want to look at the proportion of right choices (CCW) as contrast difference between left and right stimulus changes.

In [ ]:
choice, cont_left, cont_right = (behavior.TrialSet.Trial & key).fetch('trial_response_choice', 
                                                                      'trial_stim_contrast_left', 
                                                                      'trial_stim_contrast_right')


In [ ]:
signed_contrasts = cont_right - cont_left

In [ ]:
right_choices = choice == 'CCW'

In [ ]:
unique_signed_contrasts = np.unique(signed_contrasts)

total_trials = []
right_trials = []

for cont in unique_signed_contrasts:
    matching = (signed_contrasts == cont)
    total_trials.append(np.sum(matching))
    right_trials.append(np.sum(right_choices[matching]))
    

In [ ]:
prop_right_trials = np.divide(right_trials, total_trials)

In [ ]:
fig, ax = plt.subplots(1, 1, dpi=150)
ax.plot(unique_signed_contrasts * 100, prop_right_trials)
ax.set_xlabel('Signed Contrast (%)')
ax.set_ylabel('P(right choice)')

Now we have information on how choice changes across signed contrasts, let's try fitting psychometric curves. Function for fitting psychometric curves are given in `ibl_pipeline.utils`

In [ ]:
from ibl_pipeline.utils import psychofit as psy

In [ ]:
pars, L = psy.mle_fit_psycho(
        np.vstack([unique_signed_contrasts * 100, total_trials, prop_right_trials]),
        P_model='erf_psycho_2gammas',
        parstart=np.array([np.mean(unique_signed_contrasts), 20., 0.05, 0.05]),
        parmin=np.array([np.min(unique_signed_contrasts), 0., 0., 0.]),
        parmax=np.array([np.max(unique_signed_contrasts), 100., 1, 1]))

In [ ]:
x = np.linspace(-100, 100)
y = psy.erf_psycho_2gammas(pars, x)

In [ ]:
fig, ax = plt.subplots(1, 1, dpi=150)
ax.plot(unique_signed_contrasts * 100, prop_right_trials)
ax.plot(x, y)
ax.set_xlabel('Signed Contrast (%)')
ax.set_ylabel('P(right choice)')

### Bonus round - using DataJoint all the way through

Turns out you can perform all of the above steps purely within DataJoint query:

In [ ]:
trials = behavior.TrialSet.Trial & key

t_info = trials.proj('trial_response_choice', signed_contrast='trial_stim_contrast_right - trial_stim_contrast_left')
q = dj.U('signed_contrast').aggr(t_info, n='count(*)', n_right='sum(trial_response_choice="CCW")')
result = q.proj('n', 'n_right', 'signed_contrast', prop_right='n_right / n')

right_trials, total_trials, prop_right_trials, signed_contrasts = result.fetch('n_right', 'n', 'prop_right', 'signed_contrast')

In [ ]:
fig, ax = plt.subplots(1, 1, dpi=150)
ax.plot(signed_contrasts * 100, prop_right_trials)
ax.set_xlabel('Signed Contrast (%)')
ax.set_ylabel('P(right choice)')

This now gives us enough information to understand how `PsychResults` table is implemented

### PsychResults

In [ ]:
PsychResults()

In [ ]:
PsychResults.describe()

Here is an exact replica of PsychResults class definition as found in `ibl_pipeline`.

```python
class PsychResults(dj.Computed):
    definition = """
    -> behavior.TrialSet
    ---
    performance:            float   # percentage correct in this session
    performance_easy=null:  float   # percentage correct of easy trials in this session
    signed_contrasts:       blob    # contrasts used in this session, negative when on the left
    n_trials_stim:          blob    # number of trials for each contrast
    n_trials_stim_right:    blob    # number of reporting "right" trials for each contrast
    prob_choose_right:      blob    # probability of choosing right, same size as contrasts
    threshold:              float
    bias:                   float
    lapse_low:              float
    lapse_high:             float
    """

    def make(self, key):

        trials = behavior.TrialSet.Trial & key
        psych_results_tmp = utils.compute_psych_pars(trials)
        psych_results = {**key, **psych_results_tmp}

        performance_easy = utils.compute_performance_easy(trials)
        if performance_easy:
            psych_results['performance_easy'] = performance_easy

        n_trials, n_correct_trials = (behavior.TrialSet & key).fetch1(
            'n_trials', 'n_correct_trials')
        psych_results['performance'] = n_correct_trials/n_trials
        self.insert1(psych_results)
```

Most of the logic is inside the `compute_psych_pars` utility function:

```python
def compute_psych_pars(trials):

    trials = trials.proj(
        'trial_response_choice',
        signed_contrast='trial_stim_contrast_right \
        - trial_stim_contrast_left')
    q_all = dj.U('signed_contrast').aggr(trials, n='count(*)')
    q_right = dj.U('signed_contrast').aggr(
        trials, n_right='sum(trial_response_choice="CCW")')
    signed_contrasts, n_trials_stim = q_all.fetch(
        'signed_contrast', 'n'
    )
    signed_contrasts = signed_contrasts.astype(float)
    n_trials_stim = n_trials_stim.astype(int)
    n_trials_stim_right = q_right.fetch('n_right').astype(int)
    prob_choose_right = np.divide(n_trials_stim_right,
                                  n_trials_stim)

    
    # convert to percentage and fit psychometric function
    contrasts = signed_contrasts * 100
    pars, L = psy.mle_fit_psycho(
        np.vstack([contrasts, n_trials_stim, prob_choose_right]),
        P_model='erf_psycho_2gammas',
        parstart=np.array([np.mean(contrasts), 20., 0.05, 0.05]),
        parmin=np.array([np.min(contrasts), 0., 0., 0.]),
        parmax=np.array([np.max(contrasts), 100., 1, 1]))

    return {
        'signed_contrasts': signed_contrasts,
        'n_trials_stim': n_trials_stim,
        'n_trials_stim_right': n_trials_stim_right,
        'prob_choose_right': prob_choose_right,
        'bias': pars[0],
        'threshold': pars[1],
        'lapse_low': pars[2],
        'lapse_high': pars[3]
    }
```